# 기능 1 : 단어 추출에서 context 를 가진 조합으로

![alt text](img_june15.png "딥서치 참고자료")

저번에 보인 기능을 참고해보면 뉴스에서 특정 키워드들을 추출했다. 이것을 한단계 더 나아가서 특정 단일 키워드가 아닌, 여러 키워드의 조합을 추출할려고 시도해보았다.

In [1]:
from konlpy.tag import Mecab
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim import corpora
import gensim
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib
from pandas.plotting import register_matplotlib_converters
import networkx as nx

## 시각화를 위한 library
import matplotlib.pyplot as plt
import seaborn as sns

import text_helpers as t_helper

from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


sns.set()
register_matplotlib_converters()
plt.rcParams["font.family"] = 'AppleGothic'
plt.rcParams["font.size"] = 20
plt.rcParams["figure.figsize"] = (14,4)
matplotlib.rcParams['axes.unicode_minus'] = False


news_db = pd.read_csv('news_title_fixed.csv')

stock_info_path = "stock_info_0615.csv"
stock_data = pd.read_csv(stock_info_path, index_col='Unnamed: 0')




In [57]:
def get_top_topic_news(code1, code2, news_db, stock_info_db, N, print_result=False):

    name1 = stock_info_db[stock_info_db['short_code']==code1]['korean_name'].values[0]
    name2 = stock_info_db[stock_info_db['short_code']==code2]['korean_name'].values[0]

    name1 = name1.lower()
    name2 = name2.lower()    
    
    stock_news = news_db[news_db['GICODE']==code1].reset_index(drop=True)

    # text array format 변경 #
    documents = t_helper.pd_foramt_to_documents(stock_news,0,1)

    # Count-vectorizer 로 적용 #
    tf = CountVectorizer(max_features = 50, ngram_range=(3,3))
    tf_sp = tf.fit_transform(documents)

    # 점수별로 word 리스트 만들기
    tf_dict = tf.get_feature_names()
    data_array = tf_sp.toarray()

    # 이 기사들로 한해서 word score (tf_based) 로 진행
    data_array_select = data_array

    count_scores = np.sum(data_array_select,axis=0)
    count_scores_sort = np.sort(count_scores)[::-1]

    score_list = []
    for i in np.unique(count_scores_sort):
        index_list = np.argwhere(count_scores==i)
        for index in index_list:
            score_list.append((i,tf_dict[index[0]]))

    top_key_words = [word for i, word in score_list[::-1]]

    for word, score in zip(top_key_words, count_scores_sort):
        if print_result:
            print('%s : %s'%(word,score))

        ## 리셋한번 시켜주기
        stock_news['counting'] = 0

        index = np.argwhere(np.array(tf_dict)==word)[0][0]

        stock_news['counting'] = data_array[:,index]

        selected_news =stock_news[stock_news['counting']>0]

        if print_result:
            print(selected_news[:N]['TITLE'])
            print('\n\n')

    return top_key_words, count_scores_sort



In [58]:
key_words, news_info = get_top_topic_news('A021240', 'A251270', news_db, stock_data,5,True)

넷마블 웅진코웨이 인수 : 24
787    [특징주]와이제이엠게임즈, 넷마블 웅진코웨이 인수 후보자 급부상에 강세
788                 곳간 2.4조 넷마블, 웅진코웨이 인수 가능성↑
789          넷마블 웅진코웨이 인수 참전 소식에…웅진 웅진씽크빅 `대박`
794       [단독] 넷마블, 웅진코웨이 품는다…14일 인수 우선협상자로 선정
800                  넷마블, 웅진코웨이 인수 우선협상대상자로 선정
Name: TITLE, dtype: object



어워드 연속 수상 : 18
171     코웨이, 獨 'iF 디자인 어워드' 12년 연속 수상
172         코웨이, iF 디자인 어워드 12년 연속 수상
229    웅진코웨이, ‘레드닷 디자인 어워드’ 13년 연속 수상
230    웅진코웨이, '레드닷 디자인 어워드' 13년 연속 수상
232    웅진코웨이, `레드닷 디자인 어워드` 13년 연속 수상
Name: TITLE, dtype: object



디자인 어워드 연속 : 18
171     코웨이, 獨 'iF 디자인 어워드' 12년 연속 수상
172         코웨이, iF 디자인 어워드 12년 연속 수상
229    웅진코웨이, ‘레드닷 디자인 어워드’ 13년 연속 수상
230    웅진코웨이, '레드닷 디자인 어워드' 13년 연속 수상
232    웅진코웨이, `레드닷 디자인 어워드` 13년 연속 수상
Name: TITLE, dtype: object



웅진코웨이 교체 매트리스 : 12
661             웅진코웨이, '탑퍼교체 매트리스 하이브리드3' 출시
662             웅진코웨이, '탑퍼교체 매트리스 하이브리드3' 출시
663               웅진코웨이, 탑퍼교체 매트리스 하이브리드3 출시
664             웅진코웨이, 탑퍼교체 매트리스 '하이브리드3' 출시
666    <신제품ㆍ신기술>웅진코웨이, ‘탑퍼교체 매트리스 하이브리드3’ 출시
Na

현재 모델의 단점 / 보완해야될 사항 : 단어 / 문장간의 유사도를 측정할 수 없다. (Word2Vec / Doc2Vec 을 통해서 문장/단어/특정 토픽등을 벡터화 시켜야함)

-> 참고로 위의 모델의 경우

(어워드 연속 수상 / 디자인 어워드 연속) 이 두개가 같다는것을 인식하지 못한다.

단어의 경우 임베딩을 통해서 유사도를 측정하기 쉽지만, 이게 단어가 아닌 문장으로 넘어갈시에 다른 기법이 필요하다.


예시 : 비데 / 의류청정기 등은 모두 제품인데, 위의 경우 이러한 정보를 알 수가 없다.





# 기능 2 : 뉴스 이외의 분야에서 데이터 추출하기



![alt text](dart_example.png "DART 기업 공시")

DART 기업 분기별 보고서의 '사업의 내용' 항목을 통해서 각 기업들의 특징등을 추출해보았다.


In [64]:
import pickle

# Getting back the objects:
with open('dart_text.pkl','rb') as f: 
    docs = pickle.load(f)

In [65]:
print(docs[0][:1000])

 II. 사업의 내용 1. 사업의 개요 가. 사업부문별 현황 당사는 본사를 거점으로 한국 및 CE, IM 부문 산하 해외 9개 지역총괄과 DS 부문 산하 해외 5개 지역총괄의 생산ㆍ판매법인, Harman 산하 종속기업 등 244개의 종속기업으로 구성된 글로벌 전자 기업입니다. 사업군별로 보면 Set사업에서는 TV를 비롯하여 모니터, 냉장고, 세탁기 등을 생산ㆍ판매하는 CE 부문과 스마트폰 등 HHP, 네트워크시스템, 컴퓨터 등을 생산ㆍ판매하는 IM 부문이 있습니다. 부품사업에서는 DRAM, NAND Flash, 모바일AP 등의 제품을 생산ㆍ판매하고 있는 반도체 사업과 모바일ㆍTVㆍ모니터ㆍ노트북 PC용 등의 OLED 및 TFT-LCD 디스플레이 패널을 생산ㆍ판매하고 있는 DP 사업의 DS 부문으로 구성되어 있습니다. 또한, 2017년 중 인수한 Harman 부문에서 디지털 콕핏, 텔레메틱스, 스피커 등을 생산ㆍ판매하고 있습니다.   [부문별 주요 제품] 부문 주요 제품 CE 부문  TV, 모니터, 냉장고, 세탁기, 에어컨 등 IM 부문  HHP, 네트워크시스템, 컴퓨터 등 DS부문 반도체 사업  DRAM, NAND Flash, 모바일AP 등 DP 사업  OLED 스마트폰 패널, LCD TV 패널, 모니터 패널 등 Harman 부문  디지털 콕핏, 텔레메틱스, 스피커 등 지역별로 보면 국내에는 CE, IM 부문 및 반도체 사업을 총괄하는 본사와 28개의 종속기업이 사업을 운영하고 있습니다. 본사는 수원, 구미, 기흥, 화성, 평택, 광주사업장 등으로 구성되어 있으며, 국내 종속기업은 디스플레이 패널을 생산하는 삼성디스플레이㈜와 국내 대리점 판매를 전담하는 삼성전자판매㈜, 제품의 서비스를 담당하는 삼성전자서비스㈜ 및 제품의 운송을 담당하는 삼성전자로지텍㈜ 등 총 28개의 비상장 종속기업으로 구성되어 있습니다.   해외는 미주, 유럽, 아시아, 아프리카 등지에서 생산, 판매, 연구활동을 담당하는 216개의 해외 종속기업이 운영되고 있습니다. 미주는 TV, HHP 등 

In [66]:
Documents = []

## 형태소분석화 시키기
for paragraph in docs:

    # 문서본문을 형태소분석 시키기
    word_list = t_helper.article_2_word(paragraph)

    words = ''
    for word in word_list:
        words += word
        words += ' '


    # 문서제목을 형태소분석 시키기
    if words =='':
        continue
    Documents.append(words[:-1])


# TF-IDF 실행 : 단어 300개 선택

# max_df : max_document frequency
# min_df : min_document frequency
tfidf = TfidfVectorizer(max_features = 300, max_df=0.3, min_df=1,ngram_range = (1, 3))
A_tfidf_sp = tfidf.fit_transform(Documents)

tfidf_dict = tfidf.get_feature_names()
data_array = A_tfidf_sp.toarray()

count_scores = np.sum(data_array,axis=0)
count_scores_sort = np.sort(count_scores)

score_list = []
for i in np.unique(count_scores_sort):
    index_list = np.argwhere(count_scores==i)
    for index in index_list:
        score_list.append((i,tfidf_dict[index[0]]))


## 결과 Visualization 1 (키워드 -> 회사 매치)

In [67]:
for i, word in enumerate(tfidf_dict):
    print(word)
    for j, pt in enumerate(data_array[:,i]):
        if pt>0.2:
            print('%s : %.5f'%(stock_data['korean_name'][j],pt))
            
    print('\n  ')

ai
NAVER : 0.30085
SK텔레콤 : 0.47080
삼성에스디에스 : 0.35213
KT : 0.26164
한화시스템 : 0.58108
현대오토에버 : 0.23824
롯데정보통신 : 0.31009

  
al
대한전선 : 0.23270
동원시스템즈 : 0.31725

  
buy
S-Oil : 0.62945
고려아연 : 0.95755
LS ELECTRIC : 0.50548
SK케미칼 : 0.24531
SK가스 : 0.33678

  
cd
NICE : 0.32480

  
cds
하나금융지주 : 0.51928

  
cell
파미셀 : 0.37754
차바이오텍 : 0.28286
두산 : 0.26283

  
chemical
롯데케미칼 : 0.49732
네패스 : 0.33532

  
china
삼성전기 : 0.27603
삼성전기 : 0.27603
CJ제일제당 : 0.30535
실리콘웍스 : 0.20761

  
cj
CJ제일제당 : 0.87615
CJ대한통운 : 0.96527

  
cln
삼성증권 : 0.94454
메리츠증권 : 0.25026
메리츠금융지주 : 0.67642

  
cny
동원F&B : 0.21697

  
company
삼성엔지니어링 : 0.28407
휠라홀딩스 : 0.38559
한화에어로스페이스 : 0.25013

  
fda
휴젤 : 0.20989
네이처셀 : 0.26180

  
for
한올바이오파마 : 0.56827

  
fy
삼성화재 : 0.26554
한국금융지주 : 0.95990

  
gan
RFHIC : 0.97438

  
gb
삼성전자 : 0.45105
SK하이닉스 : 0.59842
LG : 0.40298
KT : 0.24502
LG유플러스 : 0.72741

  
international
오리온 : 0.31392
포스코인터내셔널 : 0.49118

  
iso
삼성전기 : 0.31546
삼성전기 : 0.31546
금호석유 : 0.39069
OCI : 0.31643
동원F&B : 0.22933

  
kr
KC

### 확실히 다음과 같은 경우를 보면 기업공시를 통해 해당 기업이 어떤 느낌의 회사인지 어느정도 알 수 있다.

lcd

LG디스플레이 : 0.36798
에스에프에이 : 0.32015
동진쎄미켐 : 0.31246
이오테크닉스 : 0.26577
에스앤에스텍 : 0.51642

led

LG이노텍 : 0.41868
케이엠더블유 : 0.71493
티씨케이 : 0.31262
서울반도체 : 0.99571
서울바이오시스 : 0.99080

lng

POSCO : 0.43179
삼성중공업 : 0.22774
GS : 0.20827
대우조선해양 : 0.43750
한국가스공사 : 0.71915
팬오션 : 0.31830
포스코인터내셔널 : 0.20417
현대미포조선 : 0.29351
SK가스 : 0.35074
하림지주 : 0.25559

oled

삼성전자 : 0.25266
삼성SDI : 0.20197
LG디스플레이 : 0.71606
에스에프에이 : 0.53070
이오테크닉스 : 0.21549
톱텍 : 0.44874
덕산네오룩스 : 0.87850
실리콘웍스 : 0.68931
에스앤에스텍 : 0.30062


건강기능식품

SKC : 0.26556
녹십자 : 0.56956
헬릭스미스 : 0.30177
콜마비앤에이치 : 0.68442
빙그레 : 0.23065
서흥 : 0.68512
종근당홀딩스 : 0.24816
풀무원 : 0.56706


게임

카카오 : 0.49857
엔씨소프트 : 0.48727
넷마블 : 0.99770
펄어비스 : 0.28367
NHN : 0.97598
더블유게임즈 : 0.95967
컴투스 : 0.98840
GKL : 0.20947
웹젠 : 0.97137


대출

우리금융지주 : 0.28087
DB손해보험 : 0.22170
BNK금융지주 : 0.55222
JB금융지주 : 0.27845
DGB금융지주 : 0.22448


리튬

에코프로비엠 : 0.83358
일진머티리얼즈 : 0.29905
후성 : 0.31784
에코프로 : 0.83536
엘앤에프 : 0.82162


In [71]:
data_array[:,0]

array([1.59016682e-01, 1.05259535e-01, 0.00000000e+00, 0.00000000e+00,
       3.00847090e-01, 0.00000000e+00, 0.00000000e+00, 9.00398990e-02,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 9.12863068e-02,
       0.00000000e+00, 0.00000000e+00, 4.70799414e-01, 7.63236555e-02,
       0.00000000e+00, 3.64030313e-04, 0.00000000e+00, 0.00000000e+00,
       3.52125021e-01, 2.63740876e-02, 7.17055922e-02, 0.00000000e+00,
       8.39675108e-02, 0.00000000e+00, 4.32614293e-02, 4.32614293e-02,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.84081917e-03,
       2.84835836e-02, 0.00000000e+00, 0.00000000e+00, 7.46822279e-03,
       1.09486847e-02, 2.61643303e-01, 0.00000000e+00, 6.04403152e-03,
       2.45665438e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 2.40521589e-04, 0.00000000e+00, 2.24334729e-02,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

## 결과 Visualization 2 (회사 -> 키워드 매치)

### 다른 접근으로, 해당 회사가 어떤 키워드를 가지고 있는지도 알 수 있다.

In [77]:
for i, kr_name in enumerate(stock_data['korean_name']):
    
    print(kr_name)
    
    for word, score in zip(tfidf_dict, data_array[i,:]):
        if score >0.2:
            print('%s : %.5f'%(word,score))
    
    print('\n\n')

삼성전자
gb : 0.45105
oled : 0.25266
tv : 0.58856
디스플레이 : 0.26352
반도체 : 0.25505
세대 : 0.27782



SK하이닉스
gb : 0.59842
pc : 0.21558
반도체 : 0.66828
카메라 : 0.24741



삼성바이오로직스
계약조건 : 0.21075
금액 비중 : 0.33287
바이오 : 0.25346
바이오의약품 : 0.58871
의약품 : 0.48037
제약사 : 0.23141



셀트리온
경쟁제품 : 0.39539
관련논문 : 0.21472
기타사항 : 0.21177
의약품 : 0.21025
임상 : 0.49020
작용기전 : 0.20447
적응증 : 0.25126
제네릭 : 0.21290
항체 : 0.22047
향후계획 : 0.21514



NAVER
ai : 0.30085
개발 진행 : 0.35314
등기 : 0.46576
방송 : 0.29631
엔진 : 0.22134
영상 : 0.31583
인터넷 : 0.24638
콘텐츠 : 0.28710
클라우드 : 0.22093



LG화학
lg : 0.50686
석유화학 : 0.45309
전지 : 0.56637



삼성SDI
oled : 0.20197
디스플레이 : 0.21268
전지 : 0.86908



카카오
게임 : 0.49857
미디어 : 0.30401
콘텐츠 : 0.66220



현대차
모듈 : 0.26762
신용카드 : 0.24600
저감 : 0.31779
차량 : 0.52917
투자자 : 0.34866
플랜트 : 0.23308



LG생활건강
특허법 : 0.23031
피부 : 0.39993
화장품 : 0.80595



삼성물산
바이오 : 0.42170
바이오의약품 : 0.29984
시공 : 0.24940
주택 : 0.24130
플랜트 : 0.29984
현장 : 0.38427



SK
sk : 0.85667
에스 : 0.22133



현대모비스
모듈 : 0.31539
영상 : 0.21414
제어 : 0.23549

IndexError: index 300 is out of bounds for axis 0 with size 300

### 이 경우에도 나쁘지는 않은것을 확인할 수 있다.

#### 삼성전자
gb : 0.45105
oled : 0.25266
tv : 0.58856
디스플레이 : 0.26352
반도체 : 0.25505
세대 : 0.27782



#### SK하이닉스
gb : 0.59842
pc : 0.21558
반도체 : 0.66828
카메라 : 0.24741



#### 삼성바이오로직스
계약조건 : 0.21075
금액 비중 : 0.33287
바이오 : 0.25346
바이오의약품 : 0.58871
의약품 : 0.48037
제약사 : 0.23141



#### 셀트리온
경쟁제품 : 0.39539
관련논문 : 0.21472
기타사항 : 0.21177
의약품 : 0.21025
임상 : 0.49020
작용기전 : 0.20447
적응증 : 0.25126
제네릭 : 0.21290
항체 : 0.22047
향후계획 : 0.21514

In [78]:


mecab.pos('화학 의외의것')


NameError: name 'mecab' is not defined